In [2]:
import numpy as np
import pandas as pd
import datetime

sales_data = pd.read_excel("Sample sales data2.xlsx")

#Read Data

In [3]:
sales_data.index = pd.to_datetime(sales_data['InvoiceDate'],format='%Y-%m-%d %H:%M:%S')

time_partitions = list()

for i in range(1,sales_data.count()['InvoiceDate']):
    time_partitions.append(datetime.datetime.strptime(str(sales_data['InvoiceDate'][i]), '%Y-%m-%d %H:%M:%S'))

# function to get unique values
#def unique(list1):
#    x = np.array(list1)
#    return np.unique(x)
#unique_time_partitions = unique(time_partitions)
#for i in range(len(unique_time_partitions)):
#    months_in_dataset.append(unique_time_partitions.month)

unique_partitions_list = list()
for time_partition in time_partitions:
    if time_partition not in unique_partitions_list:
        unique_partitions_list.append(time_partition)

months_in_dataset_list = list()
for i in unique_partitions_list:
    if i.month not in months_in_dataset_list:
        months_in_dataset_list.append(i.month)

months_in_dataset_list
        

[8, 7]

In [4]:
sales1 = sales_data.groupby(pd.Grouper(freq='M')) #Group By Month

In [5]:
sales1.count()

,DimEquipmentClassID,DimProfitCenterID,DimCustomerID,DimRateTypeID,RateType,DimRevenueTypeID,RevenueType,ContractNumber,SequenceNumber,PCID,...,MeterReadingOut,MeterReadingIn,ExcessMileageCharge,NumberCreditDays,NumberCreditHours,CreditReasonCode,RateSourceMonth,RateSourceWeek,RateSourceDay,SystemOS
InvoiceDate,,,,,,,,,,,,,,,,,,,,,
2019-07-31,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,0
2019-08-31,412,412,412,412,412,412,412,412,412,412,...,411,411,411,411,411,411,411,411,411,9


In [7]:
groups = dict(list(sales1)) #Unwrapping the group into a dictionary

In [8]:
months = list()
for i,k in groups.items():
    months.append(k)



In [9]:
for i in range(len(months)):
    print(months[i])


             DimEquipmentClassID  DimProfitCenterID  DimCustomerID  \
InvoiceDate                                                          
2019-07-31                    -1                544        3528485   
2019-07-31                    -1                525          76276   
2019-07-31                    -1               1086          90471   
2019-07-31                    -1                369         272373   

             DimRateTypeID    RateType  DimRevenueTypeID  \
InvoiceDate                                                
2019-07-31              -1  UNKNOWN                    7   
2019-07-31              -1  UNKNOWN                    6   
2019-07-31              -1  UNKNOWN                    6   
2019-07-31              -1  UNKNOWN                    6   

                         RevenueType  ContractNumber  SequenceNumber  PCID  \
InvoiceDate                                                                  
2019-07-31   Equipment / Accessories        92126893          

In [10]:
for k in range(len(months)):
    exec(f'month_{k} = pd.DataFrame(months[{k}])')

In [11]:
## Objective is to assign dataframes to month variables automatically

In [12]:
#Grouping by CustomerIDs
# ALL MONTHS
g = sales_data.groupby('DimCustomerID')

In [13]:
#Creating a list of tuples which ties the Customer (via CustomerID) to their transactions (by way of DimEquipClassID)
# ALL MONTHS
customer_list = []
for customer,customerdf in g:
    customer_list.append(customer)

In [14]:
for k in range(len(months)):
    exec(f'list_of_lists_of_equipids_based_on_customer_month_{k} = list()')


for customer in customer_list:
    for k in range(len(months)):
        exec(f"for j in range(len(month_{k}.loc[month_{k}['DimCustomerID'] == customer]['DimEquipmentClassID'])):\
            list_of_lists_of_equipids_based_on_customer_month_{k}.append(month_{k}.loc[month_{k}['DimCustomerID'] == customer]['DimEquipmentClassID'].tolist())")

In [15]:
## NOTE - STACK THE LISTS OF LISTS OF THE DIFFERENT MONTHS TO BE ABLE TO RUN APRIORI ON THE ENTIRE DATASET

combined_lists = list()
count = len(months) - 1
for k in range(count,-1,-1):
    exec(f"combined_lists.extend(list_of_lists_of_equipids_based_on_customer_month_{k})")
    
len(combined_lists)

416

In [16]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(combined_lists).transform(combined_lists)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,-1,5029,5064,5100,5184,5239,5242,5246,5247,5248,...,18811,18826,19276,19321,19325,20694,21371,21390,22903,23501
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
412,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
413,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
414,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
from mlxtend.frequent_patterns import apriori

apriori(df, min_support=0.001)

,support,itemsets
0,0.516827,(0)
1,0.004808,(1)
2,0.007212,(2)
3,0.002404,(3)
4,0.019231,(4)
...,...,...
289,0.012019,"(112, 113, 114, 118)"
290,0.012019,"(112, 113, 117, 118)"
291,0.012019,"(112, 114, 117, 118)"
292,0.012019,"(113, 114, 117, 118)"


In [18]:
frequent_itemsets = apriori(df, min_support=0.001, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.516827,(-1),1
1,0.004808,(5029),1
2,0.007212,(5064),1
3,0.002404,(5100),1
4,0.019231,(5184),1
...,...,...,...
289,0.012019,"(9003, 8915, 9732, 8951)",4
290,0.012019,"(8915, 9732, 9574, 8951)",4
291,0.012019,"(9003, 8915, 9732, 9574)",4
292,0.012019,"(9003, 9732, 9574, 8951)",4


In [19]:
frequent_itemsets[ (frequent_itemsets['length'] >= 1) &
                   (frequent_itemsets['support'] >= 0.01) ]

,support,itemsets,length
0,0.516827,(-1),1
4,0.019231,(5184),1
6,0.024038,(5242),1
7,0.045673,(5246),1
11,0.014423,(5283),1
41,0.014423,(6149),1
64,0.014423,(6405),1
69,0.012019,(6461),1
82,0.012019,(7220),1
86,0.019231,(7255),1


In [20]:
import mlxtend.preprocessing
import mlxtend.frequent_patterns

#Conf

rules1=mlxtend.frequent_patterns.association_rules(
frequent_itemsets, metric="confidence",min_threshold=0.07,support_only=False)

rules1.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(5184),(-1),0.019231,0.516827,0.007212,0.375000,0.725581,-0.002727,0.773077
1,(5239),(-1),0.009615,0.516827,0.004808,0.500000,0.967442,-0.000162,0.966346
2,(5246),(-1),0.045673,0.516827,0.028846,0.631579,1.222032,0.005241,1.311470
3,(5284),(-1),0.009615,0.516827,0.007212,0.750000,1.451163,0.002242,1.932692
4,(5500),(-1),0.009615,0.516827,0.009615,1.000000,1.934884,0.004646,inf
5,(5504),(-1),0.007212,0.516827,0.007212,1.000000,1.934884,0.003484,inf
6,(5580),(-1),0.009615,0.516827,0.007212,0.750000,1.451163,0.002242,1.932692
7,(6139),(-1),0.009615,0.516827,0.007212,0.750000,1.451163,0.002242,1.932692
8,(6149),(-1),0.014423,0.516827,0.007212,0.500000,0.967442,-0.000243,0.966346
9,(6335),(-1),0.007212,0.516827,0.007212,1.000000,1.934884,0.003484,inf


In [21]:
#Lift

rules2=mlxtend.frequent_patterns.association_rules(
frequent_itemsets, metric="lift",min_threshold=0.07,support_only=False)

rules2.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(5184),(-1),0.019231,0.516827,0.007212,0.375000,0.725581,-0.002727,0.773077
1,(-1),(5184),0.516827,0.019231,0.007212,0.013953,0.725581,-0.002727,0.994648
2,(-1),(5239),0.516827,0.009615,0.004808,0.009302,0.967442,-0.000162,0.999684
3,(5239),(-1),0.009615,0.516827,0.004808,0.500000,0.967442,-0.000162,0.966346
4,(5246),(-1),0.045673,0.516827,0.028846,0.631579,1.222032,0.005241,1.311470
5,(-1),(5246),0.516827,0.045673,0.028846,0.055814,1.222032,0.005241,1.010740
6,(5284),(-1),0.009615,0.516827,0.007212,0.750000,1.451163,0.002242,1.932692
7,(-1),(5284),0.516827,0.009615,0.007212,0.013953,1.451163,0.002242,1.004399
8,(5500),(-1),0.009615,0.516827,0.009615,1.000000,1.934884,0.004646,inf
9,(-1),(5500),0.516827,0.009615,0.009615,0.018605,1.934884,0.004646,1.009160
